In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd '/content/drive/My Drive/Subjects/AIT 582 - Applications of Metadata in Complex Big Data Problems/Project'

[Errno 2] No such file or directory: '/content/drive/My Drive/Subjects/AIT 582 - Applications of Metadata in Complex Big Data Problems/Project'
/content


In [5]:
cd '/content/drive/My Drive/AIT 582'

/content/drive/My Drive/AIT 582


In [6]:
# download the data
# https://anonfile.com/fc66r6nfo6/data_zip <- go here and copy the link below
!wget https://cdn-33.anonfile.com/fc66r6nfo6/cf393331-1587784949/data.zip

--2020-05-09 20:37:48--  https://cdn-33.anonfile.com/fc66r6nfo6/cf393331-1587784949/data.zip
Resolving cdn-33.anonfile.com (cdn-33.anonfile.com)... 194.32.146.101, 2a0c:dd43:2:1::a
Connecting to cdn-33.anonfile.com (cdn-33.anonfile.com)|194.32.146.101|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://anonfile.com/fc66r6nfo6 [following]
--2020-05-09 20:37:48--  https://anonfile.com/fc66r6nfo6
Resolving anonfile.com (anonfile.com)... 104.31.84.66, 104.31.85.66, 2606:4700:3037::681f:5442, ...
Connecting to anonfile.com (anonfile.com)|104.31.84.66|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data.zip.5’

data.zip.5              [ <=>                ]   6.62K  --.-KB/s    in 0.001s  

2020-05-09 20:37:49 (6.23 MB/s) - ‘data.zip.5’ saved [6780]



In [7]:
!unzip data.zip

Archive:  data.zip
replace processed_dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/._processed_dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:
import pandas as pd

In [9]:
df = pd.read_csv('processed_dataset.csv')

df.head()

,Unnamed: 0,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Year,Month,Day,Hour,Weekday,Time_Duration(min)
0,512224,A-512230,MapQuest,201.0,2,2019-12-25 09:49:34,2019-12-25 11:03:57,41.848064,-71.062386,NaN,NaN,0.00,Accident on MA-24 near Exit 11 Padelford St.,NaN,MA-24 N,R,Berkley,Bristol,MA,02779,US,US/Eastern,KTAN,2019-12-25 09:52:00,36.0,36.0,59.0,30.20,10.00,WNW,3.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day,2019,Dec,25,9,Wed,74.0
1,512225,A-512231,MapQuest,201.0,2,2019-12-25 12:52:38,2019-12-25 13:37:03,42.660690,-71.192589,NaN,NaN,0.00,Accident on I-495 Northbound at HIgh Plain Rd.,NaN,I-93 N,R,Andover,Essex,MA,01810,US,US/Eastern,KLWM,2019-12-25 12:54:00,36.0,36.0,55.0,30.04,10.00,CALM,0.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day,2019,Dec,25,12,Wed,44.0
2,512226,A-512232,MapQuest,201.0,2,2019-12-25 20:43:26,2019-12-25 21:58:45,42.157021,-71.069061,NaN,NaN,0.00,Lane blocked due to accident on MA-24 Northbou...,NaN,MA-24 S,R,Stoughton,Norfolk,MA,02072,US,US/Eastern,KOWD,2019-12-25 20:53:00,31.0,31.0,72.0,30.16,10.00,CALM,0.0,0.0,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night,2019,Dec,25,20,Wed,75.0
3,512227,A-512233,MapQuest,201.0,2,2019-12-25 20:57:28,2019-12-25 22:07:34,42.338776,-71.276962,NaN,NaN,0.00,Accident on I-90 Eastbound at Ridgeway Rd.,NaN,Ridgeway Rd,R,Weston,Middlesex,MA,02493,US,US/Eastern,KBED,2019-12-25 20:51:00,26.0,26.0,81.0,30.04,10.00,CALM,0.0,0.0,Fair,False,False,False,False,True,False,False,False,False,False,False,False,False,Night,Night,Night,Night,2019,Dec,25,20,Wed,70.0
4,512228,A-512234,MapQuest,201.0,3,2019-12-25 07:17:49,2019-12-25 08:49:18,42.379570,-83.200256,NaN,NaN,0.34,Accident on entry ramp to I-96 Westbound from ...,NaN,Edward J Jeffries Fwy,R,Detroit,Wayne,MI,48227,US,US/Eastern,KDET,2019-12-25 06:53:00,32.0,27.0,100.0,29.37,0.25,S,5.0,0.0,Fog,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Day,Day,2019,Dec,25,7,Wed,91.0


In [0]:
import math
ngFunctionsPresent = True
UNDEFINED_STR = "undefined"
UTMEasting = None
UTMNorthing = None
UTMZone = None     
zoneNumber = None
FOURTHPI    = math.pi / 4
DEG_2_RAD   = math.pi / 180
RAD_2_DEG   = 180.0 / math.pi
BLOCK_SIZE  = 100000
GRIDSQUARE_SET_COL_SIZE = 8   
GRIDSQUARE_SET_ROW_SIZE = 20
EASTING_OFFSET  = 500000.0
NORTHING_OFFSET = 10000000.0

In [0]:
k0 = 0.9996

EQUATORIAL_RADIUS = None
ECCENTRICTY_SQUARED = None
EQUATORIAL_RADIUS    = 6378137.0
ECC_SQUARED = 0.006694380023 
ECC_PRIME_SQUARED = ECC_SQUARED / (1 - ECC_SQUARED)
E1 = (1 - math.sqrt(1 - ECC_SQUARED)) / (1 + math.sqrt(1 - ECC_SQUARED))

In [0]:
def getZoneNumber(lat, lon):

	lat = float(lat)
	lon = float(lon)

	# sanity check on input
	#//////////////////////////////   /*
	#if lon > 360 or lon < -180 or lat > 90 or lat < -90:
	#	print "Bad input. lat: %s lon: %s" % (lat,lon)
	#//////////////////////////////  */

	# convert 0-360 to [-180 to 180] range
	lonTemp = (lon + 180) - int((lon + 180) / 360) * 360 - 180
	zoneNumber = int((lonTemp + 180) / 6) + 1

	# Handle special case of west coast of Norway
	if lat >= 56.0 and lat < 64.0 and lonTemp >= 3.0 and lonTemp < 12.0:
		zoneNumber = 32

	# Special zones for Svalbard
	if lat >= 72.0 and lat < 84.0:
		if lonTemp >= 0.0 and lonTemp < 9.0:
			zoneNumber = 31
		elif lonTemp >= 9.0 and lonTemp < 21.0:
			zoneNumber = 33
		elif lonTemp >= 21.0 and lonTemp < 33.0:
			zoneNumber = 35
		elif lonTemp >= 33.0 and lonTemp < 42.0:
			zoneNumber = 37

	return zoneNumber;  

In [0]:
def LLtoUTM_alt(lat,lon,utmcoords=None,zone=None):
	# utmcoords is a 2-D array declared by the calling routine
	lat = float(lat)
	lon = float(lon)

	# Constrain reporting USNG coords to the latitude range [80S .. 84N]
	#///////////////
	if (lat > 84.0 or lat < -80.0):
		return UNDEFINED_STR
	#////////////////////

	# sanity check on input - turned off when testing with Generic Viewer
	#///////////////////  /*
	#if lon > 360 or lon < -180 or lat > 90 or lat < -90:
	#	print "Bad input. lat: %s lon: %s" % (lat,lon)
	#////////////////////  */

	# Make sure the longitude is between -180.00 .. 179.99..
	# Convert values on 0-360 range to this range.
	lonTemp = (lon + 180) - int((lon + 180) / 360) * 360 - 180
	latRad = lat * DEG_2_RAD
	lonRad = lonTemp * DEG_2_RAD

	# user-supplied zone number will force coordinates to be computed in a particular zone
	zoneNumber = None
	if zone == None: 
		zoneNumber = getZoneNumber(lat, lon)
	else:
		zoneNumber = zone

	lonOrigin = (zoneNumber - 1) * 6 - 180 + 3  # +3 puts origin in middle of zone
	lonOriginRad = lonOrigin * DEG_2_RAD

	# compute the UTM Zone from the latitude and longitude
	UTMZone = "%s%s " % (zoneNumber,UTMLetterDesignator(lat))

	N = EQUATORIAL_RADIUS / math.sqrt(1 - ECC_SQUARED * math.sin(latRad) * math.sin(latRad))
	T = math.tan(latRad) * math.tan(latRad)
	C = ECC_PRIME_SQUARED * math.cos(latRad) * math.cos(latRad)
	A = math.cos(latRad) * (lonRad - lonOriginRad)

	# Note that the term Mo drops out of the "M" equation, because phi 
	# (latitude crossing the central meridian, lambda0, at the origin of the
	#  x,y coordinates), is equal to zero for UTM.
	M = EQUATORIAL_RADIUS * (( 1 - ECC_SQUARED / 4 - 3 * (ECC_SQUARED * ECC_SQUARED) / 64 - 5 * (ECC_SQUARED * ECC_SQUARED * ECC_SQUARED) / 256) * latRad - ( 3 * ECC_SQUARED / 8 + 3 * ECC_SQUARED * ECC_SQUARED / 32 + 45 * ECC_SQUARED * ECC_SQUARED * ECC_SQUARED / 1024) * math.sin(2 * latRad) + (15 * ECC_SQUARED * ECC_SQUARED / 256 + 45 * ECC_SQUARED * ECC_SQUARED * ECC_SQUARED / 1024) * math.sin(4 * latRad) - (35 * ECC_SQUARED * ECC_SQUARED * ECC_SQUARED / 3072) * math.sin(6 * latRad))

	UTMEasting = (k0 * N * (A + (1 - T + C) * (A * A * A) / 6 + (5 - 18 * T + T * T + 72 * C - 58 * ECC_PRIME_SQUARED ) * (A * A * A * A * A) / 120) + EASTING_OFFSET)

	UTMNorthing = (k0 * (M + N * math.tan(latRad) * ( (A * A) / 2 + (5 - T + 9 * C + 4 * C * C ) * (A * A * A * A) / 24 + (61 - 58 * T + T * T + 600 * C - 330 * ECC_PRIME_SQUARED ) * (A * A * A * A * A * A) / 720)))

	# added by LRM 2/08...not entirely sure this doesn't just move a bug somewhere else
	# utm values in southern hemisphere
	#  if (UTMNorthing < 0) {
	#	UTMNorthing += NORTHING_OFFSET;
	#  }

	utm = {}
	utm['e'] = UTMEasting
	utm['n'] = UTMNorthing
	utm['z'] = zoneNumber
	return "%s-%s-%s" % (UTMEasting, UTMNorthing, zoneNumber)
# end LLtoUTM


In [0]:
def LLtoUSNG(lat, lon, precision):

	lat = float(lat)
	lon = float(lon)

	# convert lat/lon to UTM coordinates
	ut = LLtoUTM_alt(lat, lon)
	utm = ut.split('-')
	try:
		UTMEasting = float(utm[0]) 
		UTMNorthing = float(utm[1])
		zoneNumber = utm[2]
	except IndexError:
		return ""

	# ...then convert UTM to USNG

	# southern hemispher case
	if lat < 0:
		# Use offset for southern hemisphere
		UTMNorthing = UTMNorthing + NORTHING_OFFSET

	USNGLetters  = findGridLetters(zoneNumber, UTMNorthing, UTMEasting)
	USNGNorthing = round(UTMNorthing) % BLOCK_SIZE
	USNGEasting  = round(UTMEasting)  % BLOCK_SIZE

	# added... truncate digits to achieve specified precision
	USNGNorthing = math.floor(USNGNorthing / math.pow(10,(5-precision)))
	USNGEasting = math.floor(USNGEasting / math.pow(10,(5-precision)))
	USNG = "%s%s %s " % (getZoneNumber(lat, lon),UTMLetterDesignator(lat),USNGLetters)

	# REVISIT: Modify to incorporate dynamic precision ?
	USNGEasting = ('%s' % int(USNGEasting)).zfill(precision)
	USNGNorthing = ('%s' % int(USNGNorthing)).zfill(precision)
	USNG = "%s %s %s" % (USNG, USNGEasting, USNGNorthing)

	return USNG
	# END LLtoUSNG() function


In [0]:
def UTMLetterDesignator(lat):
	lat = float(lat)

	if (84 >= lat) and (lat >= 72): 
		letterDesignator = 'X'
	elif (72 > lat) and (lat >= 64): 
		letterDesignator = 'W'
	elif (64 > lat) and (lat >= 56): 
		letterDesignator = 'V'
	elif (56 > lat) and (lat >= 48): 
		letterDesignator = 'U'
	elif (48 > lat) and (lat >= 40): 
		letterDesignator = 'T'
	elif (40 > lat) and (lat >= 32): 
		letterDesignator = 'S'
	elif (32 > lat) and (lat >= 24): 
		letterDesignator = 'R'
	elif (24 > lat) and (lat >= 16): 
		letterDesignator = 'Q'
	elif (16 > lat) and (lat >= 8):
		letterDesignator = 'P'
	elif ( 8 > lat) and (lat >= 0): 
		letterDesignator = 'N'
	elif ( 0 > lat) and (lat >= -8):
		letterDesignator = 'M'
	elif (-8> lat) and (lat >= -16): 
		letterDesignator = 'L'
	elif (-16 > lat) and (lat >= -24): 
		letterDesignator = 'K'
	elif (-24 > lat) and (lat >= -32): 
		letterDesignator = 'J'
	elif (-32 > lat) and (lat >= -40): 
		letterDesignator = 'H'
	elif (-40 > lat) and (lat >= -48): 
		letterDesignator = 'G'
	elif (-48 > lat) and (lat >= -56): 
		letterDesignator = 'F'
	elif (-56 > lat) and (lat >= -64): 
		letterDesignator = 'E'
	elif (-64 > lat) and (lat >= -72): 
		letterDesignator = 'D'
	elif (-72 > lat) and (lat >= -80): 
		letterDesignator = 'C'
	else: 
		letterDesignator = 'Z' # This is here as an error flag to show that the latitude is outside the UTM limits
	return letterDesignator
	# END UTMLetterDesignator() function

In [0]:
def findSet(zoneNum):
	zoneNum = int(zoneNum)
	zoneNum = zoneNum % 6

	if zoneNum == 0: 
		return 6
	elif zoneNum in range(1,6):
		return zoneNum
	else:
		return -1
	# END findSet() function

In [0]:
def findGridLetters(zoneNum, northing, easting):

	zoneNum  = int(zoneNum)
	northing = float(northing)
	easting  = float(easting)
	row = 1

	# northing coordinate to single-meter precision
	north_1m = round(northing)

	# Get the row position for the square identifier that contains the point
	while north_1m >= BLOCK_SIZE:
		north_1m = north_1m - BLOCK_SIZE
		row += 1

	# cycle repeats (wraps) after 20 rows
	row = row % GRIDSQUARE_SET_ROW_SIZE
	col = 0

	# easting coordinate to single-meter precision
	east_1m = round(easting)

	# Get the column position for the square identifier that contains the point
	while east_1m >= BLOCK_SIZE:
		east_1m = east_1m - BLOCK_SIZE
		col += 1

	# cycle repeats (wraps) after 8 columns
	col = col % GRIDSQUARE_SET_COL_SIZE

	return lettersHelper(findSet(zoneNum), row, col)
	# END findGridLetters() function 

In [0]:
def lettersHelper(set, row, col):

	# handle case of last row
	if row == 0:
		row = GRIDSQUARE_SET_ROW_SIZE - 1
	else:
		row -= 1

	# handle case of last column
	if col == 0:
		col = GRIDSQUARE_SET_COL_SIZE - 1
	else:
		col -=1     

	if set == 1:
		l1="ABCDEFGH"              # column ids
		l2="ABCDEFGHJKLMNPQRSTUV"  # row ids
		return l1[col] + l2[row]
	elif set == 2:
		l1="JKLMNPQR"
		l2="FGHJKLMNPQRSTUVABCDE"
		return l1[col] + l2[row]
	elif set == 3:
		l1="STUVWXYZ"
		l2="ABCDEFGHJKLMNPQRSTUV"
		return l1[col] + l2[row]
	elif set == 4:
		l1="ABCDEFGH"
		l2="FGHJKLMNPQRSTUVABCDE"
		return l1[col] + l2[row]
	elif set == 5:
		l1="JKLMNPQR"
		l2="ABCDEFGHJKLMNPQRSTUV"
		return l1[col] + l2[row]
	elif set == 6:
		l1="STUVWXYZ"
		l2="FGHJKLMNPQRSTUVABCDE"
		return l1[col] + l2[row]
	# END lettersHelper() function


In [0]:
df['grid_1000m'] = [LLtoUSNG(lat, lng, 2) for lat, lng in zip(df.Start_Lat, df.Start_Lng)]
df['grid_100m'] = [LLtoUSNG(lat, lng, 3) for lat, lng in zip(df.Start_Lat, df.Start_Lng)]
df['grid_10m'] = [LLtoUSNG(lat, lng, 4) for lat, lng in zip(df.Start_Lat, df.Start_Lng)]

In [0]:
df.to_csv('processed_dataset_grids.csv')

In [21]:
df.head()

,Unnamed: 0,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Year,Month,Day,Hour,Weekday,Time_Duration(min),grid_1000m,grid_100m,grid_10m
0,512224,A-512230,MapQuest,201.0,2,2019-12-25 09:49:34,2019-12-25 11:03:57,41.848064,-71.062386,NaN,NaN,0.00,Accident on MA-24 near Exit 11 Padelford St.,NaN,MA-24 N,R,Berkley,Bristol,MA,02779,US,US/Eastern,KTAN,2019-12-25 09:52:00,36.0,36.0,59.0,30.20,10.00,WNW,3.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day,2019,Dec,25,9,Wed,74.0,19T CG 28 34,19T CG 287 349,19T CG 2878 3496
1,512225,A-512231,MapQuest,201.0,2,2019-12-25 12:52:38,2019-12-25 13:37:03,42.660690,-71.192589,NaN,NaN,0.00,Accident on I-495 Northbound at HIgh Plain Rd.,NaN,I-93 N,R,Andover,Essex,MA,01810,US,US/Eastern,KLWM,2019-12-25 12:54:00,36.0,36.0,55.0,30.04,10.00,CALM,0.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day,2019,Dec,25,12,Wed,44.0,19T CH 20 25,19T CH 203 254,19T CH 2030 2546
2,512226,A-512232,MapQuest,201.0,2,2019-12-25 20:43:26,2019-12-25 21:58:45,42.157021,-71.069061,NaN,NaN,0.00,Lane blocked due to accident on MA-24 Northbou...,NaN,MA-24 S,R,Stoughton,Norfolk,MA,02072,US,US/Eastern,KOWD,2019-12-25 20:53:00,31.0,31.0,72.0,30.16,10.00,CALM,0.0,0.0,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night,2019,Dec,25,20,Wed,75.0,19T CG 29 69,19T CG 290 692,19T CG 2906 6928
3,512227,A-512233,MapQuest,201.0,2,2019-12-25 20:57:28,2019-12-25 22:07:34,42.338776,-71.276962,NaN,NaN,0.00,Accident on I-90 Eastbound at Ridgeway Rd.,NaN,Ridgeway Rd,R,Weston,Middlesex,MA,02493,US,US/Eastern,KBED,2019-12-25 20:51:00,26.0,26.0,81.0,30.04,10.00,CALM,0.0,0.0,Fair,False,False,False,False,True,False,False,False,False,False,False,False,False,Night,Night,Night,Night,2019,Dec,25,20,Wed,70.0,19T CG 12 89,19T CG 124 899,19T CG 1242 8990
4,512228,A-512234,MapQuest,201.0,3,2019-12-25 07:17:49,2019-12-25 08:49:18,42.379570,-83.200256,NaN,NaN,0.34,Accident on entry ramp to I-96 Westbound from ...,NaN,Edward J Jeffries Fwy,R,Detroit,Wayne,MI,48227,US,US/Eastern,KDET,2019-12-25 06:53:00,32.0,27.0,100.0,29.37,0.25,S,5.0,0.0,Fog,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Day,Day,2019,Dec,25,7,Wed,91.0,17T LG 18 94,17T LG 188 942,17T LG 1886 9426


In [22]:
df.columns

Index(['Unnamed: 0', 'ID', 'Source', 'TMC', 'Severity', 'Start_Time',
       'End_Time', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng',
       'Distance(mi)', 'Description', 'Number', 'Street', 'Side', 'City',
       'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code',
       'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)',
       'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight', 'Year', 'Month', 'Day', 'Hour', 'Weekday',
       'Time_Duration(min)', 'grid_1000m', 'grid_100m', 'grid_10m'],
      dtype='object')

In [23]:
df[df.State == 'CA'].groupby('grid_100m').size().sort_values(ascending=False)

grid_100m
11S LT  986 561    287
11S LT  816 584    156
11S MT  073 686    153
10S EG  557 847    152
11S LT  921 592    145
                  ... 
11S LB  159 824      1
11S LB  161 945      1
11S LB  161 953      1
11S LB  165 920      1
10S DH  511 982      1
Length: 44978, dtype: int64

In [24]:
!pip install geopandas

     |████████████████████████████████| 931kB 2.8MB/s 
     |████████████████████████████████| 14.7MB 312kB/s 
     |████████████████████████████████| 10.9MB 46.0MB/s 


In [0]:
!pip install pyshp

In [0]:
import pyshp
import shapely
import descartes

ModuleNotFoundError: ignored

In [0]:
!pip install geoplot
#geoplot.polyplot(world, figsize=(8, 4))

  Using cached https://files.pythonhosted.org/packages/a0/40/40c264002dfba4736f35c3524f6a0e7a03458fb4fcc436ce501f541359b7/geoplot-0.4.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/91/b5/6b54f40901d89f2ce30cf2d8110dd57658db3e95db79baba6706d7588691/mapclassify-2.2.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/c1/95/6a228a25a5e93ae236efe42a36659b61b1b4e7ceee75dd5bc4bab8944c44/contextily-1.0.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e5/92/fe8838fa8158931906dfc4f16c5c1436b3dd2daf83592645b179581403ad/Cartopy-0.17.0.tar.gz
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 /usr/local/lib/python3.6/dist-packages/pip/_vendor/pep517/_in_process.py get_requires_for_build_wheel /tmp/tmp1lrloqfy Check the logs for full command output.


In [28]:
df_cali = df[df.State == 'CA']
df_cali.head()

,Unnamed: 0,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Year,Month,Day,Hour,Weekday,Time_Duration(min),grid_1000m,grid_100m,grid_10m
292,512592,A-512598,MapQuest,201.0,2,2019-12-24 22:31:36,2019-12-25 00:16:13,37.980042,-121.746063,NaN,NaN,0.69,Accident on exit ramp from CA-4 Eastbound at E...,NaN,CA-4 E,R,Antioch,Contra Costa,CA,94531,US,US/Pacific,KCCR,2019-12-24 22:51:00,50.0,50.0,76.0,29.84,10.0,S,6.0,0.0,Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night,2019,Dec,24,22,Tue,105.0,10S FH 10 04,10S FH 101 043,10S FH 1012 0434
293,512593,A-512599,MapQuest-Bing,201.0,3,2019-12-24 22:32:19,2019-12-24 23:16:48,38.120399,-122.230270,NaN,NaN,0.93,Exit ramp from I-80 Northbound at Exits 32 32A...,NaN,I-80 E,R,Vallejo,Solano,CA,94591,US,US/Pacific,KAPC,2019-12-24 22:46:00,44.0,44.0,93.0,29.85,10.0,E,3.0,0.0,Cloudy,False,False,False,False,True,False,False,False,False,False,False,False,False,Night,Night,Night,Night,2019,Dec,24,22,Tue,44.0,10S EH 67 19,10S EH 674 194,10S EH 6747 1945
294,512594,A-512600,MapQuest,201.0,3,2019-12-24 22:42:06,2019-12-24 23:11:34,37.320969,-121.899612,NaN,NaN,0.90,Accident on exit ramp from I-280 Southbound at...,NaN,I-280 S,R,San Jose,Santa Clara,CA,95125,US,US/Pacific,KSJC,2019-12-24 22:53:00,47.0,43.0,83.0,29.85,10.0,SSE,9.0,0.0,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night,2019,Dec,24,22,Tue,29.0,10S EG 97 31,10S EG 974 310,10S EG 9749 3104
295,512595,A-512601,MapQuest,201.0,2,2019-12-24 22:45:41,2019-12-25 00:00:14,37.145073,-121.984749,NaN,NaN,0.00,#1 lane blocked due to accident on CA-17 North...,NaN,Santa Cruz Hwy,R,Los Gatos,Santa Clara,CA,95033,US,US/Pacific,KSJC,2019-12-24 22:53:00,47.0,43.0,83.0,29.85,10.0,SSE,9.0,0.0,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night,2019,Dec,24,22,Tue,75.0,10S EG 90 11,10S EG 901 114,10S EG 9016 1144
296,512597,A-512603,MapQuest-Bing,201.0,3,2019-12-25 00:03:35,2019-12-25 00:33:09,38.022110,-122.263451,NaN,NaN,0.32,Entry ramp to I-80 Southbound from Willow Ave ...,NaN,I-80 W,R,Hercules,Contra Costa,CA,94547,US,US/Pacific,KCCR,2019-12-25 00:53:00,49.0,46.0,77.0,29.81,10.0,S,7.0,0.0,Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night,2019,Dec,25,0,Wed,30.0,10S EH 64 08,10S EH 646 085,10S EH 6464 0852


In [0]:
df_cali.to_csv('cali_dataset.csv')